[//]: # (------------------------------------------    DO NOT MODIFY THIS    ------------------------------------------)
<style type="text/css">
.tg  {border-collapse:collapse;
      border-spacing:0;
     }
.tg td{border-color:black;
       border-style:solid;
       border-width:1px;
       font-family:Arial, sans-serif;
       font-size:14px;
       overflow:hidden;
       padding:10px 5px;
       word-break:normal;
      }
.tg th{border-color:black;
       border-style:solid;
       border-width:1px;
       font-family:Arial, sans-serif;
       font-size:14px;
       font-weight:normal;
       overflow:hidden;
       padding:10px 5px;
       word-break:normal;
      }
.tg .tg-fymr{border-color:inherit;
             font-weight:bold;
             text-align:left;
             vertical-align:top
            }
.tg .tg-0pky{border-color:inherit;
             text-align:left;
             vertical-align:top
            }
[//]: # (--------------------------------------------------------------------------------------------------------------)

[//]: # (-------------------------------------    FILL THIS OUT WITH YOUR DATA    -------------------------------------)
</style>
<table class="tg">
    <tbody>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Title:</td>
        <td class="tg-0pky">Notebook for model demonstrations - OneHealth Platform project</td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Authors:</td>
        <td class="tg-0pky">
            <a href="https://github.com/ecarrenolozano" target="_blank" rel="noopener noreferrer">Edwin Carreño</a>,
            <a href="" target="_blank" rel="noopener noreferrer">Inga Ulusoy</a>,
            <a href="" target="_blank" rel="noopener noreferrer">Harald Mack</a>
        </td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Affiliations:</td>
        <td class="tg-0pky">
            <a href="https://www.ssc.uni-heidelberg.de/en" target="_blank" rel="noopener noreferrer">Scientific Software Center</a>
        </td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Date Created:</td>
        <td class="tg-0pky">10th June, 2025</td>
      </tr>
      <tr>
        <td class="tg-fymr" style="font-weight: bold">Description:</td>
        <td class="tg-0pky">Notebook for demonstrating the usage of the python implementation of the original R model shown in `model_r.ipynb`</td>
      </tr>
    </tbody>
</table>

[//]: # (--------------------------------------------------------------------------------------------------------------)

In [ ]:
import model_backend as mdc
from pathlib import Path
import xarray as xr
import rioxarray as rioxr 
import numpy as np 

# Daily data - August 2024

## Build model instance

data is assumed to live in a directory: 

data/in

relative to the base directory of the package

In [ ]:
path = Path.cwd().parent / "data" / "in"
path

In [ ]:
data_file = "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc" 

In [ ]:
model = mdc.JModel(
    input = path / data_file, 
    output = Path.cwd().parent / 'data' / "out" / "output_map.nc", 
    r0_path= path / "R0_pip_stats.csv",
    run_mode="forbidden", 
    grid_data_baseurl="https://gisco-services.ec.europa.eu/distribution/v2/nuts",
    nuts_level =3, 
    resolution = "10M", 
    year=2024, 
)

In [ ]:
data = xr.open_dataset(
    path / data_file, chunks=None
)
data

In [ ]:
input = model.read_input_data().compute()


In [ ]:
input["t2m"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
input["t2m"].sel(valid_time = "2024-08-01").plot(cmap="viridis")

## Run model -> load data -> apply model transformations -> save back to netcdf

In [ ]:
model.run()

## Have a look at produced data

In [ ]:
data = xr.open_dataset(
            Path.cwd().parent / 'data' / "out" / "output_map.nc", engine="rasterio"
        )

In [ ]:
data

In [ ]:
data["R0"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
data["R0"].sel(valid_time="2024-08-01").plot(cmap="viridis")

## Run model with parallel run mode on a global scale

In [ ]:
model = mdc.JModel(
    input = path / data_file, 
    output = Path.cwd().parent / 'data' / "out" / "output_map_global.nc", 
    r0_path= path / "R0_pip_stats.csv",
    run_mode="parallelized",
    grid_data_baseurl=None,
    nuts_level =None,
    resolution = None,
    year=None,
    
)

In [ ]:
input = model.read_input_data().compute()
input["t2m"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
model.run()

In [ ]:
data = xr.open_dataset(
            Path.cwd().parent / 'data' / "out" / "output_map_global.nc", engine="rasterio", chunks = None
        ).compute()

In [ ]:
data["R0"].mean(dim="valid_time").plot(cmap="viridis")

In [ ]:
data["R0"].sel(valid_time = "2024-08-01").plot(cmap="viridis")

# Monthly data

## Build new model instance 

In [ ]:
data_file = "era5_data_2024_01_02_03_2t_tp_monthly_celsius_mm_resampled_05degree_trim.nc"

In [ ]:
model = mdc.JModel(
    input = path / data_file, 
    output = Path.cwd().parent / 'data' / "out" / "output_map_monthly.nc", 
    r0_path= path / "R0_pip_stats.csv",
    run_mode="forbidden", 
    grid_data_baseurl="https://gisco-services.ec.europa.eu/distribution/v2/nuts",
    nuts_level =3, 
    resolution = "10M", 
    year=2024, 
)

In [ ]:
data = model.read_input_data().compute()
data

In [ ]:
data["t2m"].mean(dim="time").plot(cmap="viridis")

In [ ]:
data["t2m"].sel(time = "2024-01-01").plot(cmap="viridis")

## run the model again

In [ ]:
model.run()

In [ ]:
data = xr.open_dataset(
            Path.cwd().parent / 'data' / "out" / "output_map_monthly.nc",engine="rasterio", chunks = None
        ).compute()

In [ ]:
data["R0"].mean(dim="time").plot(cmap="viridis")

since it's always summer in French Guiana by comparison, we have high transmissivity there, but literally nothing in europe where it's cold. 

In [ ]:
data["R0"].sel(time = "2024-01-01").plot(cmap="viridis")